In [496]:
import numpy as np
import pandas as pd
import scipy.sparse as sp
from numpy.linalg import norm
from collections import Counter, defaultdict
from scipy.sparse import csr_matrix

In [497]:
lines_train = []
lines_test = []
classes = []
categories = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

with open("data/train.dat", "r") as fh:
    lines = fh.readlines()
    for l in lines:
        lines_train.append(l.rstrip().split(' '))
    
with open("data/train.labels", "r") as fh:
    lines = fh.readlines()
    for l in lines:
        classes.append(l.rstrip())
    
with open("data/test.dat", "r") as fh:
    lines = fh.readlines()
    for l in lines:
        lines_test.append(l.rstrip().split(' '))

In [498]:
print(len(lines_train))
print(len(lines_test))
print(len(classes))

21186
5296
21186


In [499]:
# print(lines_train[:1])
# print(lines_test[:1])
# print(classes[:1])

In [500]:
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

from time import time

In [501]:
from sklearn.preprocessing import StandardScaler
X_train_std = StandardScaler().fit_transform(lines_train)
X_test_std = StandardScaler().fit_transform(lines_test)
# vectorizer = HashingVectorizer()
# X_train_std = vectorizer.transform(lines_train)
# X_test_std = vectorizer.transform(lines_test)

In [502]:
print(type(X_train_std))
print(X_train_std.shape)
print(type(X_test_std))
print(X_test_std.shape)

<type 'numpy.ndarray'>
(21186, 887)
<type 'numpy.ndarray'>
(5296, 887)


In [503]:
# from sklearn.decomposition import PCA
# pca = PCA(n_components=25)
# pca.fit(X_train_std)
# X_train_pca = pca.transform(X_train_std)
# X_test_pca = pca.transform(X_test_std)

In [504]:
from sklearn.decomposition import TruncatedSVD
svd = TruncatedSVD(n_components=30, n_iter=10, random_state=42)
svd.fit(X_train_std)
X_train_pca = svd.transform(X_train_std)
X_test_pca = svd.transform(X_test_std)

In [505]:
print(X_train_pca)
print(X_test_pca)

[[  2.86705424e+01   1.72106524e+00  -2.50534664e-01 ...,   3.16462380e-02
    7.08348239e-02  -1.83379140e-02]
 [  2.86705424e+01   3.85705164e+00   4.36661196e+00 ...,   1.14984057e-01
   -1.01723168e-02   5.48227309e-02]
 [  2.86705424e+01  -9.69232395e+00  -3.26048524e+00 ...,  -2.01424366e-01
    7.05864824e-02   1.02195643e-01]
 ..., 
 [  2.86705424e+01  -3.83621228e+00  -4.72358943e+00 ...,   1.34625537e-02
    3.03190263e-01   6.84452291e-02]
 [  2.86705424e+01  -1.08585838e+01  -2.17295332e+00 ...,  -5.45009950e-01
   -3.93588729e-01  -3.49016904e-01]
 [  2.86705424e+01  -2.02996547e+00  -4.83510568e+00 ...,  -1.62523348e-01
    7.57988058e-02   8.26159794e-02]]
[[ -1.04637016e+00   4.58987522e+00  -2.39569287e+00 ...,  -1.27088418e-01
    6.14520217e-02  -1.17874872e-02]
 [ -1.04637016e+00  -8.97980057e+00  -5.23357622e-01 ...,   1.70590314e-01
   -2.05146247e-01   5.09779166e-02]
 [ -1.04637016e+00  -7.05852548e+00   1.29715189e+00 ...,   2.85546332e-01
    1.44935570e-02  -

In [506]:
# vectorizer = HashingVectorizer(stop_words='english')
# X_train = vectorizer.transform(lines_train)
# X_test = vectorizer.transform(lines_test)
# vectorizer = TfidfVectorizer(sublinear_tf=True, max_df=0.5, stop_words='english')
# X_train = vectorizer.fit_transform(docs_train)


y_train = np.asarray(classes)


In [532]:
# from imblearn.over_sampling import SMOTE
# smote = SMOTE(k_neighbors=2)
# X_resampled, y_resampled = smote.fit_sample(X_train_pca, y_train)
# # X_res_vis = pca.transform(X_resampled)
# print(type(X_resampled))

In [557]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(ratio="minority")
X_resampled, y_resampled = ros.fit_sample(X_train_pca, y_train)

In [558]:
# from imblearn.over_sampling import ADASYN 
# ada = ADASYN(random_state=10, n_neighbors=2)
# X_resampled, y_resampled = ada.fit_sample(X_train_pca, y_train)



In [559]:
print(X_resampled.shape)
print(y_resampled.shape)

(31558, 30)
(31558,)


In [560]:
# target_names = ["1", "2", "3", "4", "5"]
def classify(clf, X_train, Y_train, X_test):
    print('_' * 80)
    print("Training: ")
    print(clf)
    t0 = time()
    clf.fit(X_train, Y_train)
    train_time = time() - t0
    print("train time: %0.3fs" % train_time)

    t0 = time()
    pred = clf.predict(X_test)
    test_time = time() - t0
    print("test time:  %0.3fs" % test_time)
    print(type(pred))
    return pred

In [561]:
# pred1 = classify(KNeighborsClassifier(n_neighbors=4), X_train_pca, y_train, X_test_pca)
pred1 = classify(KNeighborsClassifier(n_neighbors=4), X_resampled, y_resampled, X_test_pca)
# pred1 = classify(SVC(gamma=2), X_resampled, y_resampled, X_test_pca)
# pred1 = classify(AdaBoostClassifier(), X_resampled, y_resampled, X_test_pca)

________________________________________________________________________________
Training: 
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=4, p=2,
           weights='uniform')
train time: 0.177s
test time:  3.496s
<type 'numpy.ndarray'>


In [562]:
def writeToFile(pred):
    with open('output/output_KNeighbor.dat','w+') as f:
        for p in pred:
            f.write(str(p)+"\n")

In [563]:
print(type(pred1))
writeToFile(pred1)

<type 'numpy.ndarray'>


In [564]:
def readFile(filename):
    content = []
    with open(filename) as f:
        content = f.readlines()
        content = [x.strip() for x in content] 
    return content

In [565]:
def compare(p1, p2):
    match = 0
    total = len(p1)
    for i in range(0,total-1):                
        if(p1[i]==p2[i]):
            match = match+1
    per = ((float(match)/float(total))*100)
    return per
    

In [568]:
# p1 = readFile("output/output_KNeighbor1.dat")
p2 = readFile("submissions/Submission2_0.8168/output_KNeighbor.dat")
print(compare(pred1,p2))
print(compare(pred1,readFile("submissions/Submission10/output_KNeighbor1.dat")))


84.5166163142
82.2318731118


In [567]:
print(pred1[:10])
print(p2[:10])
print(len(pred1))
print(len(p2))

['1' '1' '2' '2' '1' '2' '1' '1' '1' '1']
['1', '1', '2', '2', '2', '2', '1', '1', '1', '1']
5296
5296
